# Submit to Kaggle

In [ ]:
model.load_state_dict(torch.load('/kaggle/working/best_model_train_all.pth'))

In [ ]:
import os
import pandas as pd
import argparse

path = '/kaggle/input/submittest/test'
def gen_filename():
    d={}
    images = os.listdir(path)
    sorted_filenames = sorted(images, key=lambda x: int(x.split("test")[1].split(".")[0]))
    for image in sorted_filenames:
        if image == '.DS_Store': continue
        d[image] = 0

    file = pd.DataFrame(d.keys(), columns=['ID'])
    file.to_csv('submission.csv', index=False)


gen_filename()


In [ ]:
submit = pd.read_csv('/kaggle/working/submission.csv')
submit


,ID
0,test0.jpg
1,test1.jpg
2,test2.jpg
3,test3.jpg
4,test4.jpg
...,...
4643,test4643.jpg
4644,test4644.jpg
4645,test4645.jpg
4646,test4646.jpg


In [ ]:
test_path = '/kaggle/input/submittest/test'
class TestDataset(Dataset):
    
    def __init__(self, 
                 img_dir, 
                 transforms=None):
        
        super().__init__()
        self.input_dataset = list()

        _, _, files = next(os.walk(os.path.join(img_dir)))
        for image_name in submit['ID']:
            input = [os.path.join(img_dir,image_name), image_name] 
            self.input_dataset.append(input)
        
        self.transforms = transforms

    def __len__(self):
        return len(self.input_dataset)

    def __getitem__(self, idx): 
        img = Image.open(self.input_dataset[idx][0]).convert('RGB')
        x = self.transforms(img)
        return x, self.input_dataset[idx][1]

submitset = TestDataset(test_path,transform)
submitloader = torch.utils.data.DataLoader(submitset, batch_size=32, shuffle=False)

In [ ]:
print('running ...')
y_predict = list()
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(submitloader):

        inputs = inputs.to(device)
        outputs = model(inputs)
        y_predict += list(outputs.argmax(dim=1).cpu().numpy())
print('Finished')


running ...


100%|██████████| 146/146 [03:33<00:00,  1.46s/it]

Finished


In [ ]:
len(y_predict)

4648

In [ ]:
submit_df = pd.DataFrame()
submit_df['ID'] = list(range(len(y_predict)))
submit_df['class'] = y_predict
submit_df

,ID,class
0,0,4
1,1,4
2,2,1
3,3,4
4,4,1
...,...,...
4643,4643,9
4644,4644,7
4645,4645,5
4646,4646,7


In [ ]:
submit_df.to_csv('submission86.csv', index=False)
